In [ ]:
import os
import keras
from keras.applications import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
print('Notebook run using keras:', keras.__version__)

### Model

In [ ]:
model = Xception(weights='imagenet')
model.summary()

### Preprocess 

In [ ]:
#Preprocess function
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        out.append(x)
    return np.vstack(out)

### Show images

In [ ]:
folder_path = 'images'

# List all files in the folder
image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))]

# Load and preprocess all images in the folder
images = transform_img_fn(image_files)

In [ ]:
# Change the index value for different images, /2 + 0.5 is to show the image in the correct color
plt.imshow(images[7] / 2 + 0.5)

### List Decoded Predictions

In [ ]:
preds = model.predict(images)

# Change the index value for different images
for x in decode_predictions(preds)[7]:
    print(x)

## Lime Explainer

In [ ]:
explainer = lime_image.LimeImageExplainer()

# Change the index value for different images, and its going to take a while
explanation = explainer.explain_instance(images[7].astype('double'), model.predict, top_labels=5, hide_color=0, num_samples=1000)

In [ ]:
# Change the index value to explain different labels
#temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
#temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
#temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False, min_weight=0.01)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

### Heat Map

In [ ]:
# Select the same class explained on the figures above.
ind =  explanation.top_labels[0]

#Map each explanation weight to the corresponding superpixel
dict_heatmap = dict(explanation.local_exp[ind])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 

#Plot. The visualization makes more sense if a symmetrical colorbar is used.
plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
plt.colorbar()